# Strategy Construction Tool
This is to produce trade reports that can be used in the evaluation section to backtest.

In [1]:
# get the data
# set up the models
# run the model with the parameters
# conver the model results into a trade log
# Log all of the intermediate steps

In [64]:
import copy
import pandas as pd
import company_data as cd
import financial_agent as fa
import committee_agent as ca
import importlib
from tqdm import tqdm

from utils.s3_helper import S3Helper
from datetime import datetime

import concurrent.futures


In [6]:
importlib.reload(fa)
importlib.reload(ca)

<module 'committee_agent' from '/project/committee_agent.py'>

### Collect the data needed to run the Agents

In [7]:
# Get the company reports
security_datasets = cd.SecurityData('tmp/fs', 'dow_quarterly_ltm_v2.json')
# get the news datasets
s3_helper = S3Helper('tmp/fs')
s3_helper.get_file(filename='dow_headlines.parquet', local_filename='/tmp/dow_headlines.parquet')
news_headlines = pd.read_parquet('/tmp/dow_headlines.parquet')

### Run the Agentic Models

In [8]:
financial_agent = fa.FinancialAnalystAgent()
committee_agent = ca.CommitteeAgent()

In [9]:
def run_single(security, as_of_date):
    company_data = security_datasets.get_security_all_data(as_of_date, security)
    # Time the run
    start_time = datetime.now()
    # Run the financial analyst agent
    financial_report = financial_agent.run(security_data=company_data, 
                                       news_data=news_headlines, 
                                       as_of_date=as_of_date)
    # Run the committee agent
    committee_report = committee_agent.run(senior_analyst_report=financial_report['senior_report'],
                                           financial_statement_analysis=financial_report['initial_analysis'],
                                           security_data=company_data)
    end_time = datetime.now()
    decision_dict = {
        'date': as_of_date,
        'security': security,
        'earning_decision': financial_report['final_output'].direction,
        'earning_magnitude': financial_report['final_output'].magnitude,
        'earning_confidence': financial_report['final_output'].confidence,
        'recommendation': committee_report['results'].recommendation,
        
        'responses': {'financial_analyst': financial_report,
                     'committee_report': committee_report},
        'time': str(end_time - start_time)
    }
    return decision_dict
    
    

In [10]:
dates_and_securities = security_datasets.date_security_timeseries()
test_security = dates_and_securities[0]

In [11]:
test_security

{'date': '2020-04-24', 'security': 'AXP UN Equity'}

In [19]:
test_output = run_single('AXP UN Equity', '2020-04-24')

In [22]:
data_output = []
def backtest(dates_and_securities):
    # create the securities
    progress = tqdm(total=len(dates_and_securities), position=0, leave=True)
    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(run_single, single['security'], single['date']) for single in dates_and_securities]
        for f in concurrent.futures.as_completed(futures):
            progress.update(n=1)
            data_output.append(f.result())
        

In [23]:
# Run the backtest
backtest(dates_and_securities)

100%|██████████| 893/893 [1:32:19<00:00,  6.20s/it]


In [27]:
for single in dates_and_securities:
    x = security_datasets.get_security_all_data(single['date'], single['security'])

In [45]:
import copy

#str(data_output[0]['responses']['committee']['final_output'])

In [59]:
data_analysis_final = copy.deepcopy(data_output)

In [47]:
data_analysis_final[0]['responses']['committee_report']['results']

Recommendation(recommendation='SELL', reason="After carefully analyzing multiple perspectives, the majority of the committee leans towards a SELL recommendation. The key factors include persistent negative profitability (operating margin of -2.52% and net profit margin of -3.00%), a dramatic 40.36% revenue decline, and signs of operational inefficiency. While there are some positive indicators like gross margin improvement, the underlying structural challenges outweigh potential recovery signals. The stock's significant price drop and continued market challenges suggest protecting investor capital through an immediate sell decision.", confidence='High')

In [44]:
data_output[0]['recommendation']

'SELL'

In [60]:
for item in data_analysis_final:
    item['responses']['financial_analyst']['final_output'] = str(item['responses']['financial_analyst']['final_output'])
    item['responses']['committee_report']['results'] = str(item['responses']['committee_report']['results'])
    item['responses']['committee_report']['history'] = str(item['responses']['committee_report']['history'])
    item['time']=str(item['time'])

In [62]:
#str(data_output[0]['responses']['committee_report']['history'])

In [25]:
import json

In [61]:
with open('/tmp/agentic_output.json', 'w') as f:
    json.dump(data_analysis_final, f)

In [63]:
s3_helper.add_file(local_filename='/tmp/agentic_output.json')